In [1]:
import numpy as np
import pandas as pd
import constants
import functions
from functions import *
from constants import *
from importlib import reload

In [2]:
consants = reload(constants)
functions = reload(functions)

In [3]:
Po = np.zeros([ndt+1, ndx, ndy])
Pw = np.zeros([ndt+1, ndx, ndy])
Po[0] = Poi

In [4]:
Qo = np.zeros([ndt+1, ndx, ndy])
Qw = np.zeros([ndt+1, ndx, ndy])
Qo[0] = qo
Qw[0] = qw

In [5]:
S_wr = 0.2
S_or = 0.1
S_wi = 0.2

In [6]:
S_w = np.zeros([ndt+1, ndx, ndy])
S_o = np.zeros([ndt+1, ndx, ndy])
S_w[0] = S_wi
S_o[0] = 1 - S_wi

In [7]:
PHI = np.zeros([ndt, ndx, ndy])
Pc = np.zeros([ndt, ndx, ndy])
dPc = np.zeros([ndt, ndx, ndy])
dBo = np.zeros([ndt, ndx, ndy])
dBw = np.zeros([ndt, ndx, ndy])
ToW = np.zeros([ndt, ndx, ndy])
ToE = np.zeros([ndt, ndx, ndy])
ToN = np.zeros([ndt, ndx, ndy])
ToS = np.zeros([ndt, ndx, ndy])
TwW = np.zeros([ndt, ndx, ndy])
TwE = np.zeros([ndt, ndx, ndy])
TwN = np.zeros([ndt, ndx, ndy])
TwS = np.zeros([ndt, ndx, ndy])
Cpoo = np.zeros([ndt, ndx, ndy])
Cpow = np.zeros([ndt, ndx, ndy])
Cswo = np.zeros([ndt, ndx, ndy])
Csww = np.zeros([ndt, ndx, ndy])
Alpha = np.zeros([ndt, ndx, ndy])
Vis_o = np.zeros([ndt, ndx, ndy])
Vis_w = np.zeros([ndt, ndx, ndy])
Bo = np.zeros([ndt, ndx, ndy])
Bw = np.zeros([ndt, ndx, ndy])
t = 0

In [8]:
Pc[0], dPc[0] = capillary_pressure(S_wr, S_or, S_w[0], ndx, ndy)

Pw[0] = Po[0] - Pc[0]

In [9]:
Vis_o[0], Vis_w[0] = viscosity(Po[0], Pw[0], ndx, ndy)

In [10]:
Bo[0], Bw[0], dBo[0], dBw[0] = FVF(Po, Pw, Bo_0, Bw_0, Co, Cw, P_0, ndx, ndy, t)
Kr_o[0], Kr_w[0] = relative_permeability(S_wr, S_or, S_w[0], ndx, ndy)
lambda_o, lambda_w = mobility(BCW, BCE, BCN, BCS, Kr_o, Kr_w, Vis_o, Vis_w, Bo, Bw, Po[0], Pw[0], ndx, ndy)

In [11]:
Jo = 2 * 3.14 * 1.127 * np.multiply(Kx, dz, lambda_o[0]) / (np.log(r_eq/rw_o) + S_o[0])
Jw = 2 * 3.14 * 1.127 * Kx * dz * (Bo / Bw * lambda_o[0] + lambda_w[0]) / (np.log(r_eq/rw_w) + S_w[0])
Jow = 2 * 3.14 * 1.127 * np.multiply(Kx, dz, lambda_w[0]) / (np.log(r_eq/rw_o) + S_o[0])

/tmp/ipykernel_4248/2077796487.py:1: RuntimeWarning: divide by zero encountered in true_divide
  Jo = 2 * 3.14 * 1.127 * np.multiply(Kx, dz, lambda_o[0]) / (np.log(r_eq/rw_o) + S_o[0])
/tmp/ipykernel_4248/2077796487.py:2: RuntimeWarning: divide by zero encountered in true_divide
  Jw = 2 * 3.14 * 1.127 * Kx * dz * (Bo / Bw * lambda_o[0] + lambda_w[0]) / (np.log(r_eq/rw_w) + S_w[0])
/tmp/ipykernel_4248/2077796487.py:3: RuntimeWarning: divide by zero encountered in true_divide
  Jow = 2 * 3.14 * 1.127 * np.multiply(Kx, dz, lambda_w[0]) / (np.log(r_eq/rw_o) + S_o[0])


In [12]:
if IBo == 1 and IBw == 1:
    BHPo[0] = Po[0] + Qo[0] / Jo
    BHPw[0] = Pw[0] + Qw[0] / Jw
    Qwo = Qo[0] * lambda_w[0] / lambda_o[0]
    for i in range(ndx):
        for j in range(ndy):
            if Qo[0][i][j] == 0:
                BHPo[0][i][j] = 0
            if Qw[0][i][j] == 0:
                BHPw[0][i][j] = 0
elif IBo == 1 and IBw == 2:
    BHPo[0] = Po[0] + Qo[0] / Jo
    Qw[0] = Jw * (-Pw[0] + BHPw[0])
    Qwo = Qo[0] * lambda_w[0] / lambda_o[0]
    for i in range(ndx):
        for j in range(ndy):
            if Qo[0][i][j] == 0:
                BHPo[0][i][j] = 0
            if BHPw[0][i][j] == 0:
                Qw[0][i][j] = 0
elif IBo == 2 and IBw == 1:
    BHPw[0] = Pw[0] + Qw[0] / Jw
    Qo[0] = (-Jo) * (Po[0] + BHPo[0])
    Qwo = (-Jow) * (Pw[0] - BHPo[0])
    for i in range(ndx):
        for j in range(ndy):
            if BHPo[0][i][j] == 0:
                Qo[0][i][j] = 0
            if Qw[0][i][j] == 0:
                BHPw[0][i][j] = 0
elif IBo == 2 and IBw == 2:
    Qo[0] = (-Jo) * (Po[0] - BHPo[0])
    Qw[0] = (Jw) * (-Pw[0] + BHPw[0])
    Qwo = (-Jow) * (Pw[0] - BHPo[0])
    for i in range(ndx):
        for j in range(ndy):
            if BHPo[0][i][j] == 0:
                Qo[0][i][j] = 0
            if BHPw[0][i][j] == 0:
                Qw[0][i][j] = 0


/tmp/ipykernel_4248/2161056674.py:22: RuntimeWarning: invalid value encountered in true_divide
  BHPw[0] = Pw[0] + Qw[0] / Jw


In [13]:
A = np.zeros([ndt+1, ndx*ndy, ndx*ndy])
b = np.zeros([ndt, ndx*ndy])

In [17]:
for n in range(1, 3):
    t += dt
    print(n)
    
    PHI[n-1] = porosity(Po[n], Cf, porosity_0, P_0, ndx, ndy)
    Pc[n], dPc[n] = capillary_pressure(S_wr, S_or, S_w[n], ndx, ndy)
    Vis_o[n], Vis_w[n] = viscosity(Po[n], Pw[n], ndx, ndy)
    Bo[n], Bw[n], dBo[n], dBw[n] = FVF(Po[n], Pw[n], Bo_0, Bw_0, Co, Cw, P_0, ndx, ndy, t)
    Kr_o[n], Kr_w[n] = relative_permeability(S_wr, S_or, S_w[n], ndx, ndy)
    lambda_o, lambda_w = mobility(BCW, BCE, BCN, BCS, Kr_o[n], Kr_w[n],\
                                  Vis_o[n], Vis_w[n], Bo[n], Bw[n], Po[n], Pw[n], ndx, ndy)
    lambda_o_west, lambda_o_east, lambda_o_north, lambda_o_south = \
                        lambda_o[1], lambda_o[2], lambda_o[3], lambda_o[4]
    lambda_w_west, lambda_w_east, lambda_w_north, lambda_w_south = \
                        lambda_w[1], lambda_w[2], lambda_w[3], lambda_w[4]
    
    ToW[n-1] = 1.127 * HW * lambda_o_west
    ToE[n-1] = 1.127 * HE * lambda_o_east
    ToN[n-1] = 1.127 * HN * lambda_o_north
    ToS[n-1] = 1.127 * HS * lambda_o_south
    TwW[n-1] = 1.127 * HW * lambda_w_west
    TwE[n-1] = 1.127 * HE * lambda_w_east
    TwN[n-1] = 1.127 * HN * lambda_w_north
    TwS[n-1] = 1.127 * HS * lambda_w_south

    Cpoo[n-1] = dx * dy * dz * (1-S_w[n]) / (5.615 * dt) * (Cf * porosity_0 * np.power(Bo[n], -1) + dBo[n] * PHI[n])
    Cpow[n-1] = dx * dy * dz * S_w[n] * / (5.615 * dt) * (Cf * porosity_0 * np.power(Bw[n], -1) + dBo[n] * PHI[n])
    Cswo[n-1] = (-dx) * dy * dz * PHI[n-1] / (5.615 * dt * Bo[n])
    Csww[n-1] = dx * dy * dz * PHI[n-1] / (5.615 * dt * Bw[n] - dPc[n] * Cpow[n-1])
    alpha[n-1] = Cswo[n-1] / Csww[n-1]
    for i in range(ndx):
        for j in range(ndy):
            # 1
            if BCW == 100 and BCN == 100 and BCS == 100 and BCE == 100:
                if IBo == 1 and IBw == 1:
                    A[n][ndy*i + j + 1][ndy*i + j + 1] = \
                    (-ToW[n][i][j]) + ToN[n][i][j] + ToS[n][i][j] + ToE[n][i][j] + Cpoo[n][i][j] \
                    - alpha[n][i][j] * (TwW[n][i][j] + TwN[n][i][j] + TwS[n][i][j] + TwE[n][i][j]  + Cpow[n][i][j])
                    
                    b[n][ndy*i + j + 1] =  (-Qo[n][i][j]) + alpha[n][i][j] * (Qw[n][i][j] + Qo[n][i][j] * lambda_w[n][i][j] / lambda_o[n][i][j]) \
                    + (alpha[n][i][j] * Cpow[n][i][j] - Cpoo[n][i][j]) * Po[n][i][j] - alpha[n][i][j] * TwE[n][i][j] * Pc[n][i][j+1]\
                    + alpha[n][i][j] * (TwE[n][i][j] + TwW[n][i][j] + TwN[n][i][j] + TwS[n][i][j]) * Pc[n][i][j]\
                    - alpha[n][i][j] * TwW[n][i][j] * Pc[n][i][j-1] - alpha[n][i][j] * TwS[n][i][j] * Pc[n][i+1][j] - alpha[n][i][j] * TwN[n][i][j] * Pc[n][i-1][j]         
                
                elif IBo == 1 and IBw == 2:
                    A[n][ndy*i + j + 1][ndy*i + j + 1] = \
                    (-ToW[n][i][j]) + ToN[n][i][j] + ToS[n][i][j] + ToE[n][i][j] + Cpoo[n][i][j] \
                    - alpha[n][i][j] * (TwW[n][i][j] + TwN[n][i][j] + TwS[n][i][j] + TwE[n][i][j]  + Cpow[n][i][j]) - alpha[n][i][j] * Jw[n][i][j]
                    
                    b[n][ndy*i + j + 1] =  (-Qo[n][i][j]) + alpha[n][i][j] * (Qo[n][i][j] * lambda_w[n][i][j] / lambda_o[n][i][j] + Jw[n][i][j] * (BHPw[n][i][j] + Pc[n][i][j])) \
                    + (alpha[n][i][j] * Cpow[n][i][j] - Cpoo[n][i][j]) * Po[n][i][j] - alpha[n][i][j] * TwE[n][i][j] * Pc[n][i][j+1]\
                    + alpha[n][i][j] * (TwE[n][i][j] + TwW[n][i][j] + TwN[n][i][j] + TwS[n][i][j]) * Pc[n][i][j]\
                    - alpha[n][i][j] * TwW[n][i][j] * Pc[n][i][j-1] - alpha[n][i][j] * TwS[n][i][j] * Pc[n][i+1][j] - alpha[n][i][j] * TwN[n][i][j] * Pc[n][i-1][j]
                
                elif IBo == 2 and IBw == 1:
                    A[n][ndy*i + j + 1][ndy*i + j + 1] = \
                    (-ToW[n][i][j]) + ToN[n][i][j] + ToS[n][i][j] + ToE[n][i][j] + Cpoo[n][i][j] \
                    - alpha[n][i][j] * (TwW[n][i][j] + TwN[n][i][j] + TwS[n][i][j] + TwE[n][i][j]  + Cpow[n][i][j]) + Jo[n][i][j] - alpha[n][i][j] * Jow[n][i][j]
                    
                    b[n][ndy*i + j + 1] =  (-Jo[n][i][j]) * BHPo[n][i][j] + alpha[n][i][j] * (Qw[n][i][j] + Jow[n][i][j] * (Pc[n][i][j] + BHPo[n][i][j])) \
                    + (alpha[n][i][j] * Cpow[n][i][j] - Cpoo[n][i][j]) * Po[n][i][j] - alpha[n][i][j] * TwE[n][i][j] * Pc[n][i][j+1]\
                    + alpha[n][i][j] * (TwE[n][i][j] + TwW[n][i][j] + TwN[n][i][j] + TwS[n][i][j]) * Pc[n][i][j]\
                    - alpha[n][i][j] * TwW[n][i][j] * Pc[n][i][j-1] - alpha[n][i][j] * TwS[n][i][j] * Pc[n][i+1][j] - alpha[n][i][j] * TwN[n][i][j] * Pc[n][i-1][j] 
                
                elif IBo == 2 and IBw == 2:
                    A[n][ndy*i + j + 1][ndy*i + j + 1] = \
                    (-ToW[n][i][j]) + ToN[n][i][j] + ToS[n][i][j] + ToE[n][i][j] + Cpoo[n][i][j] \
                    - alpha[n][i][j] * (TwW[n][i][j] + TwN[n][i][j] + TwS[n][i][j] + TwE[n][i][j]  + Cpow[n][i][j]) - alpha[n][i][j] * Jw[n][i][j] + Jo[n][i][j] - alpha[n][i][j] * Jow[n][i][j]
                    
                    b[n][ndy*i + j + 1] =  (-Jo[n][i][j]) * BHPo[n][i][j] + alpha[n][i][j] * (Jw[n][i][j] * (BHPw[n][i][j] + Pc[n][i][j]) + Jow[n][i][j] * (Pc[n][i][j] + BHPo[n][i][j])) \
                    + (alpha[n][i][j] * Cpow[n][i][j] - Cpoo[n][i][j]) * Po[n][i][j] - alpha[n][i][j] * TwE[n][i][j] * Pc[n][i][j+1]\
                    + alpha[n][i][j] * (TwE[n][i][j] + TwW[n][i][j] + TwN[n][i][j] + TwS[n][i][j]) * Pc[n][i][j]\
                    - alpha[n][i][j] * TwW[n][i][j] * Pc[n][i][j-1] - alpha[n][i][j] * TwS[n][i][j] * Pc[n][i+1][j] - alpha[n][i][j] * TwN[n][i][j] * Pc[n][i-1][j]
                A[n][ndy*i + j + 1][ndy*i + j] = ToW[n][i][j] - alpha[n][i][j] * TwW[n][i][j]
                A[n][ndy*i + j + 1][ndy*i + j + 2] = ToE[n][i][j] - alpha[n][i][j] * TwE[n][i][j]
                A[n][ndy*i + j + 1][ndy*i + j + ndy] = ToS[n][i][j] - alpha[n][i][j] * TwS[n][i][j]
                A[n][ndy*i + j + 1][ndy*i + j - ndy] = ToN[n][i][j] - alpha[n][i][j] * TwN[n][i][j]
            # 2
            elif BCW < 100 and BCN == 100 and BCS == 100 and BCE == 100:
                if IBo == 1 and IBw == 1:
                    
                elif IBo == 1 and IBw == 2:
                    
                elif IBo == 2 and IBw == 1:
                    
                elif IBo == 2 and IBw == 2:
                    
            # 3
            elif BCW == 100 and BCN < 100 and BCS == 100 and BCE == 100:
                if IBo == 1 and IBw == 1:
                    
                elif IBo == 1 and IBw == 2:
                    
                elif IBo == 2 and IBw == 1:
                    
                elif IBo == 2 and IBw == 2:
                    
            # 4
            elif BCW == 100 and BCN == 100 and BCS < 100 and BCE == 100:
                if IBo == 1 and IBw == 1:
                    
                elif IBo == 1 and IBw == 2:
                    
                elif IBo == 2 and IBw == 1:
                    
                elif IBo == 2 and IBw == 2:
                    
            # 5
            elif BCW == 100 and BCN == 100 and BCS == 100 and BCE < 100:
                if IBo == 1 and IBw == 1:
                    
                elif IBo == 1 and IBw == 2:
                    
                elif IBo == 2 and IBw == 1:
                    
                elif IBo == 2 and IBw == 2:
                    
            # 6
            elif BCW < 100 and BCN < 100 and BCS == 100 and BCE == 100:
                if IBo == 1 and IBw == 1:
                    
                elif IBo == 1 and IBw == 2:
                    
                elif IBo == 2 and IBw == 1:
                    
                elif IBo == 2 and IBw == 2:
                    
            # 7
            elif BCW < 100 and BCN == 100 and BCS < 100 and BCE == 100:
                if IBo == 1 and IBw == 1:
                    
                elif IBo == 1 and IBw == 2:
                    
                elif IBo == 2 and IBw == 1:
                    
                elif IBo == 2 and IBw == 2:
                    
            # 8
            elif BCW == 100 and BCN < 100 and BCS == 100 and BCE < 100:
                if IBo == 1 and IBw == 1:
                    
                elif IBo == 1 and IBw == 2:
                    
                elif IBo == 2 and IBw == 1:
                    
                elif IBo == 2 and IBw == 2:
                    
            # 9
            elif BCW == 100 and BCN == 100 and BCS < 100 and BCE < 100:
                if IBo == 1 and IBw == 1:
                    
                elif IBo == 1 and IBw == 2:
                    
                elif IBo == 2 and IBw == 1:
                    
                elif IBo == 2 and IBw == 2:
    

1


ValueError: could not broadcast input array from shape (10,10) into shape (10)